In [1]:
import random
import copy
import math
import time
import numpy as np
from gurobipy import *
inst_a = "Instances/a_example.txt"
inst_b = "Instances/b_lovely_landscapes.txt"
inst_c = "Instances/c_memorable_moments.txt"
inst_d = "Instances/d_pet_pictures.txt"
inst_e = "Instances/e_shiny_selfies.txt"

In [2]:
class Vignette ():
    idPhoto=[]
    tags = []
    orientation = ""
    def __init__(self,id,tags, ori):
        self.idPhoto=id
        self.tags = list(set(tags))
        self.orientation = ori
    def testScore(self,v2):
        tag1 = self.tags
        tag2 = v2.tags
        nbCommuns = 0
        nbDiff1 = 0
        nbDiff2 = 0
        scoreTotal=0
        for i in tag1:
            if(i in tag2):
                nbCommuns+=1
        nbDiff1 = len(tag1) - nbCommuns
        nbDiff2 = len(tag2) - nbCommuns
        scoreTotal+=min(nbCommuns,nbDiff1,nbDiff2)
        return scoreTotal
    def combineV(self,v2):
        if(v2.orientation != "V" or self.orientation != "V" or isinstance(self.idPhoto,list)):
            return False
        self.tags.extend(v2.tags)
        self.tags=list(set(self.tags))
        self.idPhoto = [self.idPhoto,v2.idPhoto]
        return True
    def isSingleV(self):
        if(self.orientation == "H" or isinstance(self.idPhoto,list)):
            return False
        return True
    def toString(self):
        return "Vignette : Id : "+str(self.idPhoto) + " Tags : " + str(self.tags) + " Orientation : "+self.orientation

class Presentation():
    listeV = []
    def __init__(self,arrayVignette=[]):
        self.listeV = arrayVignette
    def addVignette(self,v):
        self.listeV.append(v)
    def evaluate(self):
        """ Question 1.6
        Calcul du score de la presentation selon les regles defini par le projet"""
        if (len(self.listeV) <=1):
            return 0
        else:
            scoreTotal = 0
            for i in range(0, len(self.listeV)-1):
                tag1 = self.listeV[i].tags
                tag2 = self.listeV[i+1].tags
                nbCommuns = 0
                nbDiff1 = 0
                nbDiff2 = 0
                for i in tag1:
                    if(i in tag2):
                        nbCommuns+=1
                nbDiff1 = len(tag1) - nbCommuns
                nbDiff2 = len(tag2) - nbCommuns
                scoreTotal+=min(nbCommuns,nbDiff1,nbDiff2)
        return scoreTotal
    def copy(self):
        new = Presentation(copy.deepcopy(self.listeV))
        return new
    def swap(self,id1,id2):
        if(len(self.listeV)!=0):
            v1 = self.listeV[id1]
            v2 = self.listeV[id2]
            self.listeV[id1] = v2
            self.listeV[id2] = v1

In [3]:
def saveInstance(instance,fileName):
    """
    Sauvegarde une instance dans un fichier texte
    """
    if(errorInPres(instance)):
        print("Error in presentation : Same id used twice")
        return -1
    if(not ".txt"in fileName and not ".sol"in fileName):
        fileName+=".sol"
    fichier = open(fileName,"w")
    fichier.write(str(len(instance.listeV))+"\n")
    for i in instance.listeV:
        if(i.orientation == "H" or i.isSingleV()):
            fichier.write(str(i.idPhoto)+"\n")
        else:
            fichier.write(str(i.idPhoto[0])+" "+str(i.idPhoto[1])+"\n")
    fichier.close()

In [4]:
def errorInPres(pres):
    """Determine si une erreur est dans une presentation :
    Si une vignette est presente deux fois"""
    ids = []
    for i in pres.listeV:
        if(i.isSingleV or i.orientation == "H"):
            if(i.idPhoto in ids):
                print("\nError with id : "+str(i.idPhoto))
                return True
            ids.append(i.idPhoto)
        else:
            if(i.idPhoto[0] in ids):
                print("\nError with id : "+str(i.idPhoto[0]))
            if(i.idPhoto[1] in ids):
                print("\nError with id : "+str(i.idPhoto[1]))
            if(i.idPhoto[0] in ids or i.idPhoto[1] in ids):
                return True
            ids.append(i.idPhoto[0])
            ids.append(i.idPhoto[1])
    return False

In [5]:
def lireInstance(fileName,pourcent,verbose = False):
    """Exercice 1.2/1.3
    Prend un certain nombre de d'image dans une instance et en retourne deux tableaux:
    Un pour les images horizontales
    Un pour les verticales
    """
    instanceH = []
    instanceV = []
    fichier = open(fileName,"r")
    nb = int(fichier.readline())
    if(verbose):
        print(nb)
    cpt = 0
    line = fichier.readline()
    while(line != None and cpt <nb*pourcent):
        if(verbose):
            print(cpt)
            print(line)
        line = line.split(" ")
        ori = line[0]
        if(verbose):
            print("ori : "+ori)
        id = cpt
        nbTags = int(line[1])
        if(verbose):
            print("id : "+str(id))
        tags = []
        for i in range(2,len(line)):
            if("\n" in line[i]):
                tags.append(line[i][0:len(line[i])-1])
            else:
                tags.append(line[i])
            if(verbose):
                print(line[i])
        line = fichier.readline()
        if(ori == "H"):
            instanceH.append(Vignette(id,tags,ori))
        else:
            instanceV.append(Vignette(id,tags,ori))
        cpt+=1

    return instanceH,instanceV

In [6]:
def question13(inst=inst_a,pourcent = 0.1,verbose = False):
    """ Question 1.3
    lis les fichier de l'instance a_example.txt 
    Trie les photos dans deux tableau 
    Cree une presentation avec d abord les photos horizontales puis les verticales 2 a 2"""
    h,v = lireInstance(inst,pourcent)
    print("Taille h: "+str(len(h)))
    print("Taille v: "+str(len(v)))
    pres = Presentation([])
    
    #Parcours toutes les vignettes horizontalles et les ajoutes directement a la presentation
    print("H")
    for i in h:
        pres.addVignette(i)
    
    #Parcours toutes les vignettes horizontalles et les ajoutes directement a la presentation
    print("V")
    for i in range(0,len(v),2):
        if(i < len(v)-1):
            #Si cette vignette n'est pas la derniere, on en met 2 cote a cote
            vi = Vignette([v[i].idPhoto,v[i+1].idPhoto],v[i].tags+v[i+1].tags,"V")
        else:
            vi = v[i]
        
        if(verbose):
            print("Add vignette "+vi.toString())
        pres.addVignette(vi)

    print("Presentation : Taille "+str(len( pres.listeV)))
    #for i in pres.listeV:
        #print(i.toString())
        
    print("Fin Pres")

    print("Score : "+str(pres.evaluate()))
    saveInstance(pres,"resultat.txt")

In [7]:
question13(inst_c,1)

Taille h: 500
Taille v: 500
H
V
Presentation : Taille 750
Fin Pres
Score : 152


In [22]:
def glouton(instanceH,instanceV,verbose = False):
    """Prend en entree deux ensembles de vignettes (V et H)
    Tant que les deux instances ne sont pas vides 
    On analyse toutes les vignettes disponible et on insert la plus efficace (par rapport au score de la transition)
    """
    
    pres = Presentation([])
    #Choix de la première vignette : une chance sur deux de prendre une horizontale 
    if(random.uniform(0, 1)>0.5 and len(instanceH) != 0):
        iVignette = int(random.uniform(0,len(instanceH)))
        pres.addVignette(instanceH[iVignette])
        instanceH.remove(instanceH[iVignette])
    else:
        iVignette = int(random.uniform(0,len(instanceV)))
        iVignette2 = iVignette
        while(iVignette == iVignette2):
            iVignette2 = int(random.uniform(0,len(instanceV)))
        v = instanceV[iVignette]
        v2 = instanceV[iVignette2]
        v.combineV(v2)
        instanceV.remove(v)
        instanceV.remove(v2)
        pres.addVignette(v)
    
    
    while(len(instanceH)>0 or len(instanceV)>0):#Tant que les deux instance ne sont pas vides
        if(verbose):
            print(len(instanceH))
            print(len(instanceV))
        score = -1
        vType =""
        idV = -1
        idV2 = -1
        #On parcours les vignettes horizontales et on récupère celle qui donne le meilleur score
        for i in range(0,len(instanceH)):
            test = pres.listeV[len(pres.listeV)-1].testScore(instanceH[i])
            if(test> score or idV == -1):
                score = test
                vType = "H"
                idV = i
        #On teste toutes les vignettes verticales 2 a 2 et on garde la meilleure combinaison 
        for i in range(0,len(instanceV)):
            for j in range(0,len(instanceV)):
                if(i!= j):
                    v = Vignette(instanceV[i].idPhoto,instanceV[i].tags,instanceV[i].orientation)
                    v.combineV(instanceV[j])
                    test = pres.listeV[len(pres.listeV)-1].testScore(v)
                    if(test> score or idV == -1):
                        score = test
                        vType = "V"
                        idV = i
                        idV2 = j
        
        #On ajoute la meilleurs vignettes (h ou v) a la présentation
        if(vType == "H"):
            pres.addVignette(instanceH[idV])
            instanceH.remove(instanceH[idV])
        elif(vType == "V"):
            v = Vignette(instanceV[idV].idPhoto,instanceV[idV].tags,instanceV[idV].orientation)
            v.combineV(instanceV[idV2])
            if(verbose):
                print("\n####")
                print("id2"+str(idV2))
                print(str(instanceV[idV2].idPhoto))
                print(v.toString()+"\n")
            delV1 = instanceV[idV]
            delV2 = instanceV[idV2]
            instanceV.remove(delV1)
            instanceV.remove(delV2)
            pres.addVignette(v)
            if(verbose):
                print("####\n")
        elif(V == -1 and len(instanceV)!=0):
            pres.addVignette(instanceV[0])
            instanceV.remove(instanceV[0])
    return pres
            
    

In [23]:
def gloutonOpti(instanceH,instanceV,nbARegarder,verbose = False):
    """Prend en entree deux ensembles de vignettes (V et H)
    Tant que les deux instances ne sont pas vides 
    On analyse un nombre restreint de vignettes et on insert la plus efficace (par rapport au score de la transition)"""
    pres = Presentation([])
    #Choix de la première vignette : une chance sur deux de prendre une horizontale 
    if(random.uniform(0, 1)>0.5 and len(instanceH) != 0):
        iVignette = int(random.uniform(0,len(instanceH)))
        pres.addVignette(instanceH[iVignette])
        instanceH.remove(instanceH[iVignette])
    else:
        iVignette = int(random.uniform(0,len(instanceV)))
        iVignette2 = iVignette
        while(iVignette == iVignette2):
            iVignette2 = int(random.uniform(0,len(instanceV)))
        v = instanceV[iVignette]
        v2 = instanceV[iVignette2]
        
        v.combineV(v2)
        instanceV.remove(v)
        instanceV.remove(v2)
        pres.addVignette(v)
        
    while(len(instanceH)>0 or len(instanceV)>0):#Tant que les deux instance ne sont pas vides
        score = -1
        vType =""
        V = -1
        V2 = -1
        #On parcours x vignettes horizontales et on récupère celle qui donne le meilleur score
        if(len(instanceH) == 0):
            vignetteAEtudierH = []
        else:
            vignetteAEtudierH = random.sample(instanceH,max(1,min(len(instanceH),nbARegarder)))
        
        for i in vignetteAEtudierH:
            test = pres.listeV[len(pres.listeV)-1].testScore(i)
            if(test> score or V == -1):
                score = test
                vType = "H"
                V = i
        
        vignetteAEtudierV = random.sample(instanceV,min(len(instanceV),nbARegarder))
        
        #On teste toutes les vignettes verticales(parmis un nombre restreint) 2 a 2 et on garde la meilleure combinaison 
        for i in vignetteAEtudierV:
            for j in vignetteAEtudierV:
                if(i!= j):
                    v = Vignette(i.idPhoto,i.tags,i.orientation)
                    v.combineV(j)
                    test = pres.listeV[len(pres.listeV)-1].testScore(v)
                    if(test> score or V == -1):
                        score = test
                        vType = "V"
                        V = i
                        V2 = j
        
        #On ajoute la meilleurs vignettes (h ou v) a la présentation            
        if(vType == "H"):
            pres.addVignette(V)
            instanceH.remove(V)
        elif(vType == "V"):
            v = Vignette(V.idPhoto,V.tags,V.orientation)
            v.combineV(V2)
            instanceV.remove(V)
            instanceV.remove(V2)
            pres.addVignette(v)
        elif(V == -1 and len(instanceV)!=0):
            pres.addVignette(instanceV[0])
            instanceV.remove(instanceV[0])
        else:
            print("Rien a faire")
            print(V)
            print(len(instanceH))
            print(len(instanceV))
    return pres
            
    

In [25]:
def scorePermutation(pres,idV,idNewV,verbose = False):
    """Recupere une presentation et deux vignettes
    Calcule le score de la presentation lorsque les deux vignettes sont echanges"""
    if(verbose):
        print(len(pres.listeV))
        print(idV)
        print(idNewV)
    newP = pres.copy()
    newP.swap(idV,idNewV)
    return newP.evaluate()

In [24]:
def descente_stochastiques(pres,it,nbVoisinnages,choixRandom):
    """Recupere une presentation deja faite et test differentes permutations pour ameliorer le score"""
    nbSwap =0
    if(choixRandom):#On choisit une vignette au hasard pour tester différentes permutations
        cpt = 0
        while cpt < it:#Effectue des permutations it fois
            cpt+=1
            #On choisit une vignette au hasard a permuter
            idV = int(random.uniform(0,len(pres.listeV)-1))
            bestEvalScore = pres.evaluate()
            
            idSwap1 = -1
            idSwap2 = -1
            #On analyse un certain nombre de voisin
            for i in range(0,nbVoisinnages):
                #On recupere au hasard une vignette qui a la meme orientation que notre vignette a permuter
                idV2 = idV
                antiblock = 0
                while((idV2 == idV or pres.listeV[idV].orientation != pres.listeV[idV2].orientation) and antiblock < 10):
                    antiblock+=1
                    idV2 = int(random.uniform(0,len(pres.listeV)-1))
                if(antiblock < 10):#Empeche une recherche de vignette trop longue
                    continue
                    
                #On calcule le score de la permutation
                newScore = scorePermutation(pres,idV,idV2)
                
                #Si la permutation est meilleure, on la garde
                if(bestEvalScore < newScore):
                    bestEvalScore = newScore
                    idSwap1 = idV
                    idSwap2 = idV2
            if(idSwap1 != -1 and idSwap2 != -1):
                pres.swap(idSwap1,idSwap2)
                nbSwap+=1
    else:#Toutes les vignettes pourront etre permutées
        for idV in range(0,len(pres.listeV)-1):#On parcours toutes les vignettes
            bestEvalScore = pres.evaluate()
            idSwap1 = -1
            idSwap2 = -1
            
            #On analyse un certain nombre de voisin
            for i in range(0,nbVoisinnages):
                #On recupere au hasard une vignette qui a la meme orientation que notre vignette a permuter
                idV2 = idV
                antiblock = 0
                while((idV2 == idV or pres.listeV[idV].orientation != pres.listeV[idV2].orientation) and antiblock < 10):
                    antiblock+=1
                    idV2 = int(random.uniform(0,len(pres.listeV)-1))
                if(antiblock < 10):
                    continue
                    
                #On calcule le score de la permutation
                newScore = scorePermutation(pres,idV,idV2)
                
                #Si la permutation est meilleure, on la garde
                if(bestEvalScore < newScore):
                    bestEvalScore = newScore
                    idSwap1 = idV
                    idSwap2 = idV2
            if(idSwap1 != -1 and idSwap2 != -1):
                pres.swap(idSwap1,idSwap2) 
                nbSwap+=1
    
    print("Nombre d'échange : "+str(nbSwap))
    return pres

In [26]:
def trouverVoision(pres,nbIter,nbVoisinEtu):
    """Recupere une presentation, tente differentes permutations qu'on garde si elles ameliorent le score"""
    pre = pres.copy()
    cpt = 0
    #On effectue 'nbIter' test de permutations
    while cpt < nbIter:
        cpt+=1
        
        #On choisit une vignette au hasard a permuter
        idV = int(random.uniform(0,len(pre.listeV)))
        bestEvalScore = pre.evaluate()
        idSwap1 = -1
        idSwap2 = -1
        
        #On analyse un certain nombre de voisin
        for i in range(0,nbVoisinEtu):
            
            #On recupere au hasard une vignette qui a la meme orientation que notre vignette a permuter
            idV2 = idV
            antiblock = 0
            while((idV2 == idV or pre.listeV[idV].orientation != pre.listeV[idV2].orientation) and antiblock < 10):
                antiblock+=1
                idV2 = int(random.uniform(0,len(pres.listeV)))
            if(antiblock < 10):
                continue
                
            #On calcule le score de la permutation
            newScore = scorePermutation(pres,idV,idV2)
            
            #Si la permutation est meilleure, on la garde
            if(bestEvalScore < newScore):
                bestEvalScore = newScore
                idSwap1 = idV
                idSwap2 = idV2
        if(idSwap1 != -1):
            pre.swap(idSwap1,idSwap2)
            return pre
    return pre

In [27]:
def recuit(pres,nbIter = 10,probInit=2):
    """Tente d'ameliorer une presentation en analysants differentes permutations
    Recuit : une permutation peux être garder meme si elle n'est pas meilleure pour analyser un plus grand voisinnage"""
    prob = probInit
    score = pres.evaluate()
    it = 0
    nbSwap=0
    while (it<nbIter):
        it+=1
        newPres = trouverVoision(pres.copy(),10,min(len(pres.listeV),25))
        newScore = newPres.evaluate()
        if(score < newScore or random.uniform(0, 1)< math.exp(-(score-newScore)/prob)):
            nbSwap+=1
            pres = newPres
        prob = prob*0.99
    print("Nombre echange : "+str(nbSwap))
    return pres

In [47]:
h,v=lireInstance(inst_c,0.5)
pre = gloutonOpti(h,v,int(max(len(h)/100,len(v)/100)),True)
print("Score :"+str(pre.evaluate()))
pre = recuit(pre,len(pre.listeV)*10)
print("###################New###################")
print("Score :"+str(pre.evaluate()))
saveInstance(pre ,"Checker_RP/c_resultat_descenteOpti.sol")

Score :130
Nombre echange : 2970
###################New###################
Score :255


In [28]:
def randomInitiation(instanceH,instanceV):
    pres = Presentation([])
    while(len(instanceH)>0 and len(instanceV)>0):#Tant que les deux instance ne sont pas vides
        score = -1
        vType =""
        idV = -1
        idV2 = -1
        #On parcours les vignettes horizontales et on récupère celle qui donne le meilleur score
        
        if((random.uniform(0, 1)>0.5 and len(instanceH) > 0)or len(instanceV) == 0):
            idV = int(random.uniform(0,len(instanceH)))
            pres.addVignette(instanceH[idV])
            instanceH.remove(instanceH[idV])
        elif(len(instanceV)>1):
            idV = int(random.uniform(0,len(instanceV)))
            idV2 = idV
            while(idV2 == idV):
                idV2 = int(random.uniform(0,len(instanceV)))
            
            v = Vignette(instanceV[idV].idPhoto,instanceV[idV].tags,instanceV[idV].orientation)
            v.combineV(instanceV[idV2])
            delV1 = instanceV[idV]
            delV2 = instanceV[idV2]
            instanceV.remove(delV1)
            instanceV.remove(delV2)
            pres.addVignette(v)
        else:
            pres.addVignette(instanceV[0])
            instanceV.remove(instanceV[0])
            
    return pres

In [29]:
def myTechnique(instH,instV,nbIter,typeAmelio):
    pres = randomInitiation(copy.deepcopy(instH),copy.deepcopy(instV))
    for i in range(nbIter):
        pre = randomInitiation(copy.deepcopy(instH),copy.deepcopy(instV))
        if(pres.evaluate()<= pre.evaluate()):
            pres = pre
    print(pres.evaluate())
    if(typeAmelio =="R"):
        pres = recuit(pres,nbIter)
    elif(typeAmelio =="D"):
        pres = descente_stochastiques(pres,nbIter,1000,True)
    return pres

In [20]:

h,v=lireInstance(inst_b,0.01)
nbIter = 100
nbVoisin = 100
d=dict()
#Calcul temps d'execution et score du glouton
t = time.time()
pres_glouton = glouton(copy.deepcopy(h),copy.deepcopy(v)) 
print(pres_glouton.evaluate())

57


In [48]:
inst=inst_a
taille_analyse=1
h,v=lireInstance(inst,taille_analyse)
myTechnique(h,v,100,"R").evaluate()

0
Nombre echange : 100


0

In [30]:
#Analyse experimentale
def comparaison(inst,p,b=False):
    """Applique les differents algorithmes et en retire le score obtenu et le temps d'execution"""
    taille_analyse = p
    h,v=lireInstance(inst,taille_analyse)
    nbIter = 100
    nbVoisin = 100
    d=dict()
    if(b):
        print("Initialization done")
    #Calcul temps d'execution et score du glouton
    t = time.time()
    pres_glouton = glouton(copy.deepcopy(h),copy.deepcopy(v),b) 
    d["glouton"]=[pres_glouton.evaluate(),time.time()-t]
    print(d["glouton"])
    #Calcul temps d'execution et score du glouton opti
    t = time.time()
    pres_gloutonOpti = gloutonOpti(copy.deepcopy(h),copy.deepcopy(v),nbVoisin,True)
    d["gloutonOpti"]=[pres_gloutonOpti.evaluate(),time.time()-t]
    print(d["gloutonOpti"])
    #Calcul temps d'execution et score de la descente stochastiques
    t = time.time()
    pre_descente_glouton = gloutonOpti(copy.deepcopy(h),copy.deepcopy(v),nbVoisin,True)
    pre_descente = descente_stochastiques(pre_descente_glouton,nbIter,nbVoisin,True)
    d["descente"]=[pre_descente.evaluate(),time.time()-t]
    print(d["descente"])
    #Calcul temps d'execution et score du recuit simulé
    t = time.time()
    pre_recuit_glouton = gloutonOpti(copy.deepcopy(h),copy.deepcopy(v),nbVoisin,True)
    pre_recuit_glouton = recuit(pre_recuit_glouton,nbIter)
    d["recuit"]=[pre_recuit_glouton.evaluate(),time.time()-t]
    print(d["recuit"])
    #Calcul temps d'execution et score de notre technique personnelle avec Recuit
    t = time.time()
    p = myTechnique(copy.deepcopy(h),copy.deepcopy(v),100,"R")
    d["myR"]=[p.evaluate(),time.time()-t]
    print(d["myR"])
    #Calcul temps d'execution et score de notre technique personnelle avec Descente
    t = time.time()
    p = myTechnique(copy.deepcopy(h),copy.deepcopy(v),100,"D")
    d["myD"]=[p.evaluate(),time.time()-t]
    print(d["myD"])
    return d

In [31]:
a = comparaison(inst_a,1)
print(a)

[1, 0.0]
[2, 0.0]
Nombre d'échange : 0
[2, 0.4031345844268799]
Nombre echange : 100
[2, 0.05682015419006348]
1
Nombre echange : 100
[1, 0.012994766235351562]
1
Nombre d'échange : 0
[1, 2.7989227771759033]
{'glouton': [1, 0.0], 'gloutonOpti': [2, 0.0], 'descente': [2, 0.4031345844268799], 'recuit': [2, 0.05682015419006348], 'myR': [1, 0.012994766235351562], 'myD': [1, 2.7989227771759033]}


In [ ]:

b = comparaison(inst_b,0.0005,False)
print(b)

[0, 0.0049855709075927734]


In [ ]:

c = comparaison(inst_c,0.0005)
print(c)

In [ ]:

d = comparaison(inst_d,0.0005)
print(d)

In [ ]:

e = comparaison(inst_e,0.0005)
print(e)

Nombre echange : 1000


46

In [17]:
inst= inst_c
taille_analyse = 0.1
h,v=lireInstance(inst,taille_analyse)
pres_gloutonOpti = gloutonOpti(h,v,1000,True)
print(pres_gloutonOpti.evaluate())

107


In [ ]:
def score(vignettes,i,j):
    tag1 = vignettes[i].tags
    tag2 = vignettes[j].tags
    nbCommuns = 0
    nbDiff1 = 0
    nbDiff2 = 0
    scoreTotal=0
    for i in tag1:
        if(i in tag2):
            nbCommuns+=1
    nbDiff1 = len(tag1) - nbCommuns
    nbDiff2 = len(tag2) - nbCommuns
    scoreTotal+=min(nbCommuns,nbDiff1,nbDiff2)
    return scoreTotal

In [19]:
import numpy as np
a = np.arange(0,10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [28]:
def solveurPL(vignettes):
    
    
    # Range of plants and warehouses
    
    
    
    m = Model("mogplex")     
            
    # declaration variables de decision
    x = []
    z = []
    
    
    
    for i in range(0,len(vignettes)):
        for j in range(0,len(vignettes)):
            #Une variable xi correspond a une transition entre vi et vj si == 1
            x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d" % (i*len(vignettes)+j)))
    
    for i in range(0,len(vignettes)):
        for j in range(0,len(vignettes)):
            #Variable zi : elimination des sous tours par les flots
            z.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="x%d" % (i*len(vignettes)+j)))        
    
    # maj du modele pour integrer les nouvelles variables
    print(len(x))
    print(len(z))
    m.update()
          
    # definition de l'objectif  
    obj = LinExpr();
    obj =0  
    
    # definitions des contraintes
    #Un seul arc sortant d'un sommet
    for i in range(0,len(vignettes)):
        #Empeche une transition vers la même vignette
        #m.addConstr(x[i*len(vignettes)+i] == 0)
        
        m.addConstr(quicksum(x[i*len(vignettes)+j] for j in range(0,len(vignettes)))<=1)
        m.addConstr(quicksum(x[j*len(vignettes)+i] for j in range(0,len(vignettes)))<=1)
        
     
    
    #Contrainte du premier sommet
    m.addConstr(quicksum(z[j] for j in range(1,len(vignettes))) == len(vignettes)-1)
    
    
            
    for i in range(1,len(vignettes)):
        i1 = np.arange(0,len(vignettes))
        i1=np.delete(i1,i)
        i1=np.delete(i1,0)
        i2 = np.arange(0,len(vignettes))
        i2=np.delete(i2,i)
        m.addConstr(quicksum([z[i*len(vignettes)+j] for j in i1])+1 == quicksum([z[j*len(vignettes)+i] for j in i2]))
    
    for i in range(0,len(vignettes)):
        for j in range(0,len(vignettes)):
            if(j != i and j != 0):
                m.addConstr(z[i*len(vignettes)+j]+z[j*len(vignettes)+i] <= (len(vignettes)-1)*(x[i*len(vignettes)+j]+x[j*len(vignettes)+i]))
    
    
    for i in range(0,len(vignettes)):
        for j in range(0,len(vignettes)):
            if(j != i and j != 0):
                m.addConstr(z[i*len(vignettes)+j]>=0)
    # maj du modele pour integrer les nouvelles variables
    m.update()
    
    obj = LinExpr();
    obj =0
    for i in range(0,len(vignettes)):
        for j in range(0,len(vignettes)):
            obj+= score(vignettes,i,j)*x[i*len(vignettes)+j]
            #obj+= x[i*len(vignettes)+j]
    # definition de l'objectif
    m.setObjective(obj,GRB.MAXIMIZE)
    print("PL SET")
    # Definition des contraintes
    
    # Resolution
    m.optimize()
    
    
    print("")                
    print('Solution optimale:')
    print("")
    return x

In [22]:

inst= inst_b

taille_analyse = 0.0003

h,v=lireInstance(inst,taille_analyse)
print(len(v))
print(len(h))
pres_glouton = gloutonOpti(h,v,10) 
pres_glouton.evaluate()

0
24


0

In [23]:

inst= inst_b

taille_analyse = 0.0003

h,v=lireInstance(inst,taille_analyse)
print(len(v))
print(len(h))
pres_glouton = glouton(h,v) 
pres_glouton.evaluate()

0
24
23
0
22
0
21
0
20
0
19
0
18
0
17
0
16
0
15
0
14
0
13
0
12
0
11
0
10
0
9
0
8
0
7
0
6
0
5
0
4
0
3
0
2
0
1
0


0

In [30]:

inst= inst_b

taille_analyse = 0.0003

h,v=lireInstance(inst,taille_analyse)
print(len(h))
a =solveurPL(h)

24
576
576
PL SET
Optimize a model with 1130 rows, 1152 columns and 4855 nonzeros
Variable types: 576 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 782 rows and 71 columns
Presolve time: 0.01s
Presolved: 348 rows, 1081 columns, 3243 nonzeros
Variable types: 529 continuous, 552 integer (552 binary)

Root relaxation: objective -0.000000e+00, 177 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.00000    0   10          -   -0.00000      -     -    0s
     0     0   -0.00000    0   21          -   -0.00000      -     -    0s
*    0     0               0      -0.0000000   -0.00000  0.00%     -    0s

Cutting planes:
  Gomory: 2
  MIR: 1
  GUB cover: 5

Explored 1 nodes (2285 simplex